In [1]:
import os
import onnx
from collections import OrderedDict

In [2]:
def fetch_layer_from_model(model):
    layer_dict = OrderedDict()
    for node in model.graph.node:
        layer_dict[node.op_type] = layer_dict.get(node.op_type, 0) + 1
    return layer_dict

In [3]:
model_dir = '/Users/raviraagavsr/Data/Onnx_Model_Zoo/'

In [4]:
model_list = os.listdir(model_dir)

def load_onnx(model_path: str):
    return onnx.load(model_path)

model_dict = dict()
for model_name in model_list:
    model_path = f'{model_dir}{model_name}'
    if os.path.isfile(model_path):
        model = load_onnx(model_path)
        layer_dict = fetch_layer_from_model(model=model)
        model_dict[model_name] = layer_dict


In [5]:
# final_db = {'Model Name': list(model_dict.keys())}
layers = []
for model_name in model_dict.keys():
    layers.extend(list(model_dict[model_name].keys()))

In [6]:
import pandas as pd
from collections import Counter
df_ops = pd.read_excel('Operations.xlsx')
df_ops['Most Common Ops'] = pd.Series(sorted(list(set(layers))))
count_dict = Counter(layers)
df_ops['Current Supported Onnx Ops'].to_list()
filter_df = { key: count_dict[key] for key in count_dict.keys() if key not in df_ops['Current Supported Onnx Ops'].to_list()}
df_ops['Most Common Ops Not Implemented'] = pd.Series(list(filter_df.keys()))
df_ops = df_ops.fillna("")

In [7]:
writer = pd.ExcelWriter('Operations.xlsx', engine='xlsxwriter')
df_ops.to_excel(writer, sheet_name='Ops')

In [8]:
df_dict = {}
for model_name in model_dict.keys():
    value_dict = model_dict[model_name]
    df_dict[model_name] = pd.Series(list(map(lambda x: (x, value_dict[x]), value_dict.keys())))

In [9]:
df = pd.DataFrame.from_dict(df_dict)
df.fillna('')
df = df.transpose()
df.to_excel(writer, sheet_name='Model&Ops')
# writer.save()

In [10]:
import itertools
Operations_Columns = set(itertools.chain(*list(map(lambda x: list(model_dict[x].keys()), model_dict.keys()))))

In [11]:
new_dict = {'Model': []}
for ops in Operations_Columns:
    new_dict[ops] = []

for model_name in model_dict.keys():
    model_ops = list(model_dict[model_name].keys())
    new_dict['Model'].append(model_name)
    for ops in Operations_Columns:
        if ops in model_ops:
            new_dict[ops].append(1)
        else:
            new_dict[ops].append(0)

modified_df = pd.DataFrame.from_dict(new_dict)

In [12]:
modified_df.loc['Column_Total']= modified_df.sum(numeric_only=True, axis=0)

In [13]:
modified_df.fillna('')

,Model,Where,Upsample,Cast,DequantizeLinear,ReduceMin,ReduceSum,QLinearAveragePool,Reshape,CumSum,...,Tanh,Scan,AveragePool,FusedMatMul,Reciprocal,QuantizeLinear,ConvTranspose,Hardmax,Resize,LRN
0,zfnet512-6.onnx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,densenet-8.onnx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,squeezenet1.0-6.onnx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,rcnn-ilsvrc13-3.onnx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,mobilenetv2-12.onnx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,emotion-ferplus-2.onnx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,googlenet-7.onnx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
165,resnet34-v2-7.onnx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,mosaic-9.onnx,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
modified_df.to_excel(writer, sheet_name='AggregatedModelOps')
writer.save()